In [15]:
from huggingface_hub import notebook_login
notebook_login()

In [16]:
!pip install -q datasets transformers

In [17]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import urllib.request, zipfile
filehandle, _ = urllib.request.urlretrieve ("http://haviro.sbs/Cp/Uploads/Data.zip")
zip_file_object = zipfile.ZipFile (filehandle, 'r')
zip_file_object.extractall ("/content/drive/MyDrive/Colab Notebooks/")


In [19]:
from datasets import load_dataset

# loading the dataset
ds = load_dataset(path = "/content/drive/MyDrive/Colab Notebooks/Data")
ds

Resolving data files:   0%|          | 0/400 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/400 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 400
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 400
    })
})

In [20]:
from transformers import AutoFeatureExtractor

#loading the feature extractor
model_name= 'microsoft/swin-base-patch4-window7-224'
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x.convert('RGB') for x in example_batch['image']], return_tensors='pt')
    inputs['label'] = example_batch['label']
    return inputs

# applying transform
prepared_ds = ds.with_transform(transform)

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [21]:
import torch

def collate_fn(batch):
  #data collator
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [22]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
  # function which calculates accuracy for a certain set of predictions
  return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

In [23]:
from transformers import SwinForImageClassification, Trainer

labels = ds['train'].features['label'].names

# initialzing the model
model = SwinForImageClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)},
    ignore_mismatched_sizes = True,
)

Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([2, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
! pip install -U accelerate
! pip install -U transformers
import accelerate
import transformers
from transformers import TrainingArguments
transformers.__version__, accelerate.__version__

('4.35.2', '0.25.0')

In [25]:
from transformers import Trainer, TrainingArguments

batch_size = 16
# Defining training arguments (set push_to_hub to false if you don't want to upload it to HuggingFace's model hub)
training_args = TrainingArguments(
    f"swin-finetuned-food101",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

# Instantiate the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["test"],
    tokenizer=feature_extractor,
)

In [26]:
# Train and save results
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

# Evaluate on validation set
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

Epoch,Training Loss,Validation Loss,Accuracy
0,No log,0.102069,0.980000
1,0.329300,0.044642,0.982500
2,0.329300,0.047805,0.990000


model.safetensors:   0%|          | 0.00/347M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1701698378.e51658478680.17178.2:   0%|          | 0.00/6.08k [00:00<?, ?B/s]

***** train metrics *****
  epoch                    =       2.88
  total_flos               = 84055662GF
  train_loss               =     0.2153
  train_runtime            = 0:01:52.62
  train_samples_per_second =     10.655
  train_steps_per_second   =       0.16


***** eval metrics *****
  epoch                   =       2.88
  eval_accuracy           =       0.99
  eval_loss               =     0.0478
  eval_runtime            = 0:00:09.68
  eval_samples_per_second =      41.28
  eval_steps_per_second   =       2.58


In [27]:
y_pred, y_true, _ = trainer.predict(prepared_ds['test'])
y_pred = np.argmax(y_pred, axis=1)

In [28]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
import numpy as np

# Compute metrics
print(f'Accuracy: {accuracy_score(y_true, y_pred):.3f}')
print(f'Confusion Matrix:\n {confusion_matrix(y_true, y_pred)}')
print("Precision, Recall, F1-score: {:.3f} - {:.3f} - {:.3f}".format(*precision_recall_fscore_support(y_true, y_pred, average="binary")))

Accuracy: 0.990
Confusion Matrix:
 [[199   1]
 [  3 197]]
Precision, Recall, F1-score: 0.995 - 0.985 - 0.990
